In [1]:
using constants
using types
using Optim
using PyPlot
using data
include("DOSfunctions.jl")
include("userfunctions.jl")
include("propertyfuncs.jl")

LoadError: LoadError: syntax: invalid type signature
while loading /home/abdullah/working_directory/jupyter_wd/julia/scienctific/BTE/types.jl, in expression starting on line 8

In [2]:
Ts=collect(150.01:50.0:800.01)
#Ts=collect(663.8:0.01:664.0)#[663.0,663.1]
numofn=[-1.5e20,-7.3e19,-2.0e19,-0.15e19]#,-0.2e20]#,-1.75e20,-3.0e19,-0.9e20]#Sample 1 -3.0e19, Sample 2 -1.25e20,
#Sample 3 -1.75e20, Sample Sb 0.75% -1.03e20, #
numofnn=[1e17,1e17,1e17,1e17]#,1e17]
xs=[0.287,0.3,0.3,0.3]#[0.287,0.3,0.262,0.332]#collect(0.0:0.01:1.0)

var0=[Ts[1],xs[1]]
Eoff=5.0
Ev(var)=Eoff
Ec(var)=Ev(var)+EgSiGe(var[1],var[2]) #var[1]:T var[2]:x
alphanp=0.0#0.2#1/EgSiGe([Ts[1],xs[1]])


effMassV(var)=-1.2*me
effMassC(var)=1.4/6^(2/3)*me#0.321me#0.424*me#0.424*me#1.4/6^(2/3)*me

bandC=parBandTx(effMassC(var0),alphanp,Ec(var0),[Ec],[effMassC],var0)
bandV=parBandTx(effMassV(var0),0.0,Ev(var0),[Ev],[effMassV],var0)
#bndstTx=BandStrucTx([bandV,bandC,bandC,bandC,bandC,bandC,bandC],var0)
bndstTx=BandStrucTx([bandC,bandC,bandC,bandC,bandC,bandC,bandV],var0)#,bandC,bandC,bandC,bandC,bandC,bandV],var0)
#var array: var[1]: effMass , var[2]:T, var[3]:E, var[4]:x, var[5]:Ef, 
#var[6]:band, var[7]:NII var[8]:bndst var[9]:NNI var[10]:bandV

md=effMassC([Ts[1],xs[1]])*6^(2/3)#1.4*me;
function Deftemp(band::parBandTx)
    Da=7.0
    Dv=3.8
    if band==bandC
        return [Da,Dv]#10.5#15.0
    end
    if band==bandV        
        return [Dv,Da]#7.0
    end
end
ksi=21.0
tauAC(var)=tauAC_func(ClSiGe(var[4],var[2]),Deftemp(var[6])[1],Deftemp(var[6])[2],var[2],var[1],var[3],var[6])#DefP(var[1])
tauII2(var)=tauII2_func(ksi*eps0,eps0,var[6],var[5],var[2],var[7],var[1],var[3],var[8])
tauNI(var)=tauNI_func(ksi*eps0,var[1],var[9])
vartau0=[bandC.effMass,Ts[1],Ec(var0),xs[1],Ec(var0),bandC,numofn[1]*1e6,bndstTx,numofnn[1]*1e6,bandV]
#vartau0=[bandC.effMass,Ts[1],1.0,xs[1],1.5,bandC,numofn[1]*1e6,bndstTx,numofnn[1]*1e6]
tauTOTTx=tau_electron_B([tauAC,tauII2,tauNI],vartau0)#tauNI,,tauII2,tauAC,
xmax=3.0
#T_EF_Dict=Dict()
#println("-3")

LoadError: UndefVarError: parBandTx not defined

In [3]:
@time (Efs,sigmas,seebecks,kes,kees,kehs,kbis)=electronicPropsiso(bndstTx,xs,Ts,numofn,tauTOTTx,xmax)

Efsr=Array{Float64,3}(length(Ts),length(numofn),length(xs))
for (i,iT) in enumerate(Ts)
    for (k,kx) in enumerate(xs) 
        for (j,jn) in enumerate(numofn)
            Efsr[i,j,k]=Efs[i,j,k]-Ec([iT,kx])
        end
    end
end
#println("-2")
T_EF_Dict=Dict()
for (Ti,Tx) in enumerate(Ts)
    T_EF_Dict[Tx]=(Efs[Ti])    
end
T_EF_Dict_Array=Array{Dict{Float64,Float64},2}(length(numofn),length(xs))
Dict_temp=Dict()
Efsr=Array{Float64,3}(length(Ts),length(numofn),length(xs))
for (j,jn) in enumerate(numofn)
    for (k,kx) in enumerate(xs) 
        for (i,iT) in enumerate(Ts)
            Efsr[i,j,k]=Efs[i,j,k]-Ec([iT,kx])
            Dict_temp[iT]=Efsr[i,j,k]
            T_EF_Dict_Array[j,k]=Dict_temp 
        end
    end
end

LoadError: UndefVarError: electronicPropsiso not defined

In [4]:
#println("-1")
#############Lattice Thermal Conductivity Parameters###############
thetaSiL=586.0 # K 
thetaSiT=240.0 # K
vSiL=8430.0    # m/s
vSiT=5840.0    # m/s
vGeL=4920.0    # m/s
vGeT=3540.0    # m/s
gammaL=0.9
gammaT=0.9
MSi=4.6637e-26 # kg
MGe=1.206e-25  # kg
VSi=2.0024e-29 #m3
VGe=2.2636e-29 #m3

Eep=3.4*q
xsp=xs[1]
Tp=300.0
p=1.0
D=20e-9
NP=0.0
a=1.0
alpha=0.5 #specularity factor
d=97.0e-9#

#println("1")

#vartauPH0=[[0.02],Ts[1],gammaL,gammaT,xsp,thetaCLx,thetaCTx,thetaCTx,MSi,MGe,VSi,a,T_EF_Dict,0.0]
vartauPH0=[ [0.02],  Ts[1],   gammaL,  gammaT, xsp,
    thetaSiL, thetaSiT, thetaSiT, MSi,   MGe, 
    VSi, a, T_EF_Dict_Array, Eep, alpha, 
    d, thetaSiL, thetaSiT, vSiL, vSiT, 
    vGeL, vGeT, p, NP,Tsindex=1,
    numberofnindex=1,xsindex=1]
thetaCLx=thetaCL(vartauPH0[12],vartauPH0[5],vartauPH0[17],vartauPH0[19],vartauPH0[11])
thetaCTx=thetaCT(vartauPH0[12],vartauPH0[5],vartauPH0[18],vartauPH0[19],vartauPH0[11])
#println("2")
tauPHNLf(var)=tauPH_NL(var[3],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeL_PH(var[5]))
tauPHNTf(var)=tauPH_NT(var[4],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeT_PH(var[5]))

tauPHULf(var)=tauPH_U(var[3],var[1],var[2],MSiGe(var[5]),vSiGeL_PH(var[5]),var[17])
tauPHUTf(var)=tauPH_U(var[4],var[1],var[2],MSiGe(var[5]),vSiGeT_PH(var[5]),var[18])

tauPHALLLf(var)=tauPH_ALL(var[9],var[10],var[5],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeL_PH(var[5]))
tauPHALLTf(var)=tauPH_ALL(var[9],var[10],var[5],VSiGe(var[5]),var[1],var[2],MSiGe(var[5]),vSiGeT_PH(var[5]))

tauPHeLf(var)=tauPH_e(var[14],md,MSiGe(var[5])/VSiGe(var[5]),(var[13][var[26],var[27]])[var[2]],var[1],var[2],
vSiGeL_PH(var[5]))
tauPHeTf(var)=tauPH_e(var[14],md,MSiGe(var[5])/VSiGe(var[5]),(var[13][var[26],var[27]])[var[2]],var[1],var[2],
vSiGeT_PH(var[5]))

tauPHBLf(var)=tauPH_B(vSiGeL_PH(var[5]),var[14],var[15])
tauPHBTf(var)=tauPH_B(vSiGeT_PH(var[5]),var[14],var[15])
#println("3")
tauPHTOTL=tau_phonon_B([tauPHNLf,tauPHULf,tauPHALLLf,tauPHeLf],vartauPH0)#,tauPH_eLf ,tauPHALLLf ,tauPHBLf
tauPHNL=tau_phonon_B([tauPHNLf],vartauPH0)
tauPHRL=tau_phonon_B([tauPHULf,tauPHALLLf,tauPHeLf],vartauPH0) #,tauPHALLLf,tauPH_eLf] ,tauPHBLf

tauPHTOTTx=tau_phonon_B([tauPHNTf,tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #tauPHALLTf,tauPH_eTf ,tauPHBTf
tauPHNTx=tau_phonon_B([tauPHNTf],vartauPH0)
tauPHRTx=tau_phonon_B([tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #,tauPHALLTf,tauPH_eTf ,tauPHBTf

tauPHTOTTy=tau_phonon_B([tauPHNTf,tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #,tauPHALLTf,tauPH_eTf tauPHBTf
tauPHNTy=tau_phonon_B([tauPHNTf],vartauPH0)
tauPHRTy=tau_phonon_B([tauPHUTf,tauPHALLTf,tauPHeTf],vartauPH0) #,tauPHALLTf,tauPH_eTf ,tauPHBTf
#println("4")

tauPHL=[tauPHTOTL,tauPHNL,tauPHRL]
tauPHTx=[tauPHTOTTx,tauPHNTx,tauPHRTx]
xmax=1.0

LoadError: UndefVarError: T_EF_Dict_Array not defined

In [5]:
tau=0
tau+=1

1

In [6]:
function tauPH_U_SAT(gamma::Float64,intx::Array{Float64},T::Float64,M::Float64,
    theta::Float64,omegaD::Float64,beta::Float64,delta::Float64)
    #intx is different from x in oither phonon calculations intx=omega/omegaD
    Mcgs=M
    #println("Mcgs= ",Mcgs)
    deltacgs=delta*100
    #println("deltacgs= ",deltacgs)
    #println("1/Mcgs/deltacgs^2/(theta/T)= ",(1/Mcgs/deltacgs^2/(theta/T)))
    return 1./((3.264e-2)*((1+beta*(5/9))/(1+beta))*gamma^2.*intx.^2./Mcgs/deltacgs^2/(theta/T))    
end

function tauPH_EP_SAT(Eep::Float64,md::Float64,x::Array{Float64},Ef::Float64,
    T::Float64,M::Float64,theta::Float64,delta::Float64)
    eta=Ef#Ef*q/kB/T
    deltacgs=delta*100
    Mcgs=M
    A=6.76e26(md/me)^2*deltacgs^2/Mcgs
    y=3.72e9(md/me)*deltacgs^2*theta
    D=1.68e-11/(md/me)/deltacgs^2/theta
    alphat=thetaD/T
    #println("alphat ",alphat)
    lambda=6
    ex1=1+exp(-alphat*y+eta-D*alphat*x.*x+alphat.*x/2)
    ex2=1+exp(-alphat*y+eta-D*alphat*x.*x-alphat.*x/2)    
    ext=ex1./ex2
    logt=log(ext)
    return 1./(lambda*(A*Eep^2/alphat).*logt) 
end

function tauPH_PD_SAT(GM::Float64,intx::Array{Float64},theta::Float64)
    return 1./(6.17e11*theta*GM.*intx.^4)
end
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
function tauPH_U_VT(gamma::Float64,intx::Array{Float64},T::Float64,M::Float64,theta::Float64,
omegaD::Float64,beta::Float64,delta::Float64)#intx is different from x in oither phonon calculations intx=omega/omegaD
    bb=20*pi/3*hbar*6.02e23*(6*pi*pi/4)^(1/3)
    M=M/1000
    deltacgs=delta
    #println("bb ",bb)
    return 1./(bb*(1+beta*5/9)/(1+beta)*gamma^2/M/deltacgs^2*(T/theta)^3.*intx.^2)    
end
function tauPH_C_VT(gamma::Float64,intx::Array{Float64},T::Float64,M::Float64,theta::Float64,
omegaD::Float64,beta::Float64,delta::Float64)#intx is different from x in oither phonon calculations intx=omega/omegaD
   tau_u=tauPH_U_VT(gamma::Float64,intx::Array{Float64},T::Float64,M::Float64,theta::Float64,
omegaD::Float64,beta::Float64,delta::Float64)#intx is different from x in oither phonon calcu
    return tau_u#1./(1./(tau_u/beta)+1./tau_u)    
end
function tauPH_PD_VT(GM::Float64,Tt::Float64,intx::Array{Float64},vtt::Float64,delta::Float64)
    return 1./(GM/4/pi*(delta/vtt)^3*(kB*Tt/hbar)^4.*intx.^4)
end
function tauPH_EP_VT(Eep::Float64,md::Float64,x::Array{Float64},Ef::Float64,v::Float64,T::Float64,d::Float64)
    ts=md*v*v/(2*kB*T)
    Efst=Ef#*q/kB/T
    ex1=(1+exp(-ts+Efst-x.*x/16/ts+x/2))
    ex2=(1+exp(-ts+Efst-x.*x/16/ts-x/2))    
    return 1./((Eep^2*md^3*v)/(4*pi*hbar^4*d*ts).*log(ex1./ex2))
end

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
function tauPH_NLT(gamma::Float64,V::Float64,x::Array{Float64},T::Float64,M::Float64,v::Float64)
    
    return M*v^5*hbar^4./(kB^5*gamma^2*V*x.*x*T^5)    
end
function tauPH_UT(gamma::Float64,x::Array{Float64},T::Float64,M::Float64,v::Float64,theta::Float64)   
    return 1./((hbar*gamma^2*kB^2.*x.^2*T^3*exp(-theta/3/T)/(M*v^2*theta*hbar^2)))
end
function tauPH_ALLT(Gamma::Float64,MI::Float64,MII::Float64,y::Float64,V::Float64,
    x::Array{Float64},T::Float64,M::Float64,v::Float64)
     GammaT=(1-y)*((MI-M)/M)^2+y*((MII-M)/M)^2
    return 1./(Gamma*V/v^3./4./pi)./(kB*T*x/hbar).^4 #4*pi*v^3*hbar^4./(Gamma*V*(kB*T*x).^4)
end
function tauPH_eT(Eep::Float64,md::Float64,ro::Float64,Ef::Float64,x::Array{Float64},T::Float64,v::Float64)
    #Ef=Ef*q/kB/T
    beta=md*v^2/2/kB/T
    logp=log((1+exp(-beta+Ef-x.*x/(16*beta)+x/2))./(1+exp(-beta+Ef-x.*x/(16*beta)-x/2)))
    return 4*pi*hbar^4*ro*beta./(Eep^2*md^3*v)./(logp) 
end
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
############################################################################################################
function I1SS(gammaSA,GM,Tt,MSiGecgs,V,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
    alphat=thetaD/Tt
    M=MSiGecgs
    T=Tt
    v=vtt
    Ef=etha
    gamma=gammaSA
    ro=dd
    md=mds
    tauPH_U_SS_Af(x)=tauPH_UT(gamma,x,T,M,v,thetaD)
    tauPH_N_SS_Af(x)=tauPH_NLT(gamma,V,x,T,M,vtt)
    tauPH_EP_SS_Af(x)=tauPH_eT(Eep*q,md,ro,Ef,x,T,v)/6   
    tauPH_PD_SS_Af(x)=tauPH_ALLT(GM,4.6637e-26,1.206e-25,0.3,V,x,T,M,v)    
    tauPH_C_SS_Af(x)=1./(1./tauPH_N_SS_Af(x)+1./tauPH_U_SS_Af(x)+1./tauPH_EP_SS_Af(x)+1./tauPH_PD_SS_Af(x))    
    integrand(x)=tauPH_C_SS_Af(x).*(x.^4).*exp(x)./(exp(x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,alphat)
    return a= do_quad(integrand,nodes, weights)   
end
function I2SS(gammaSA,GM,Tt,MSiGecgs,V,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
    alphat=thetaD/Tt
    M=MSiGecgs
    T=Tt
    v=vtt
    Ef=etha
    gamma=gammaSA
    ro=dd
    md=mds
    tauPH_U_SS_Af(x)=tauPH_UT(gamma,x,T,M,v,thetaD)
    tauPH_N_SS_Af(x)=tauPH_NLT(gamma,V,x,T,M,vtt)
    tauPH_EP_SS_Af(x)=tauPH_eT(Eep*q,md,ro,Ef,x,T,v)/6   
    tauPH_PD_SS_Af(x)=tauPH_ALLT(GM,4.6637e-26,1.206e-25,0.3,V,x,T,M,v)    
    tauPH_C_SS_Af(x)=1./(1./tauPH_N_SS_Af(x)+1./tauPH_U_SS_Af(x)+1./tauPH_EP_SS_Af(x)+1./tauPH_PD_SS_Af(x)) 
    tauPH_R_SS_Af(x)=1./(1./tauPH_U_SS_Af(x)+1./tauPH_EP_SS_Af(x)+1./tauPH_PD_SS_Af(x)) 
    integrand(x)=tauPH_C_SS_Af(x)./tauPH_N_SS_Af(x).*(x.^4).*exp(x)./(exp(x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,thetaD/Tt)
    return a= do_quad(integrand,nodes, weights)    
end

function I3SS(gammaSA,GM,Tt,MSiGecgs,V,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
alphat=thetaD/Tt
    M=MSiGecgs
    T=Tt
    v=vtt
    Ef=etha
    gamma=gammaSA
    ro=dd
    md=mds
    tauPH_U_SS_Af(x)=tauPH_UT(gamma,x,T,M,v,thetaD)
    tauPH_N_SS_Af(x)=tauPH_NLT(gamma,V,x,T,M,vtt)
    tauPH_EP_SS_Af(x)=tauPH_eT(Eep*q,md,ro,Ef,x,T,v)/6   
    tauPH_PD_SS_Af(x)=tauPH_ALLT(GM,4.6637e-26,1.206e-25,0.3,V,x,T,M,v)    
    tauPH_C_SS_Af(x)=1./(1./tauPH_N_SS_Af(x)+1./tauPH_U_SS_Af(x)+1./tauPH_EP_SS_Af(x)+1./tauPH_PD_SS_Af(x)) 
    tauPH_R_SS_Af(x)=1./(1./tauPH_U_SS_Af(x)+1./tauPH_EP_SS_Af(x)+1./tauPH_PD_SS_Af(x))    
    integrand(x)=tauPH_C_SS_Af(x)./tauPH_N_SS_Af(x)./tauPH_R_SS_Af(x).*(x.^4).*exp(x)./(exp(x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,alphat)
    return a= do_quad(integrand,nodes, weights)    
end
function kltSS(gammaSA,GM,Tt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
    
    I1t=I1SS(gammaSA,GM,Tt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    I2t=I2SS(gammaSA,GM,Tt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    I3t=I3SS(gammaSA,GM,Tt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    return kB^4*Tt^3/2/pi/pi/hbar^3/vtt*(I1t+I2t.^2/I3t)    
end

###########################################################################################################
function I1(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)
    alphat=thetaD/Tt
    tauPH_U_SA_Af(x)=tauPH_U_SAT(gammaSA,x,Tt,MSiGecgs,thetaD,omegaD,beta,delta)   
    tauPH_EP_SA_Af(x)=tauPH_EP_SAT(Eep,mds,x,etha,Tt,MSiGecgs,thetaD,delta)
    tauPH_PD_SA_Af(x)=tauPH_PD_SAT(GM,x,thetaD)
    tauPH_C_SA_Af(x)=1./((beta+1)./tauPH_U_SA_Af(x)+1./tauPH_EP_SA_Af(x)+1./tauPH_PD_SA_Af(x))    
    integrand(x)=tauPH_C_SA_Af(x).*alphat.*(x.^4)*alphat^4.*exp(alphat*x)./(exp(alphat*x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,1.0)
    return a= do_quad(integrand,nodes, weights)    
end

function I2(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)
    alphat=thetaD/Tt
    tauPH_U_SA_Af(x)=tauPH_U_SAT(gammaSA,x,Tt,MSiGecgs,thetaD,omegaD,beta,delta)    
    tauPH_EP_SA_Af(x)=tauPH_EP_SAT(Eep,mds,x,etha,Tt,MSiGecgs,thetaD,delta)
    tauPH_PD_SA_Af(x)=tauPH_PD_SAT(GM,x,thetaD)
    tauPH_C_SA_Af(x)=1./((beta+1)./tauPH_U_SA_Af(x)+1./tauPH_EP_SA_Af(x)+1./tauPH_PD_SA_Af(x))    
    integrand(x)=beta*tauPH_C_SA_Af(x)./tauPH_U_SA_Af(x).*(x.^4)*alphat^5.*exp(alphat*x)./(exp(alphat*x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,1.0)
    return a= do_quad(integrand,nodes, weights)    
end

function I3(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)
    alphat=thetaD/Tt
    tauPH_U_SA_Af(x)=tauPH_U_SAT(gammaSA,x,Tt,MSiGecgs,thetaD,omegaD,beta,delta)    
    tauPH_EP_SA_Af(x)=tauPH_EP_SAT(Eep,mds,x,etha,Tt,MSiGecgs,thetaD,delta)
    tauPH_PD_SA_Af(x)=tauPH_PD_SAT(GM,x,thetaD)
    tauPH_C_SA_Af(x)=1./((beta+1)./tauPH_U_SA_Af(x)+1./tauPH_EP_SA_Af(x)+1./tauPH_PD_SA_Af(x))    
    integrand(x)=beta*1./tauPH_U_SA_Af(x).*
    (1-beta*tauPH_C_SA_Af(x)./tauPH_U_SA_Af(x)).*(x.^4)*alphat^5.*exp(alphat*x)./(exp(alphat*x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,1.0)
    return a= do_quad(integrand,nodes, weights)    
end

function klt(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)    
    alphat=thetaD/Tt
    I1t=I1(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)
    I2t=I2(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)
    I3t=I3(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha)
    return 4.67e-2*(thetaD^2/delta/100/alphat^3)*(I1t+I2t.^2/I3t)    
end
###########################################################################################################
#

function I1V(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
alphat=thetaD/Tt
    tauPH_U_V_Af(x)=tauPH_U_VT(gammaSA,x,Tt,MSiGecgs,thetaD,omegaD,beta,delta)     
    tauPH_EP_V_Af(x)=tauPH_EP_VT(Eep*q,mds,x,etha,vtt,Tt,dd)/6    
    tauPH_PD_V_Af(x)=tauPH_PD_VT(GM,Tt,x,vtt,delta)    
    tauPH_C_V_Af(x)=1./((beta+1)./tauPH_U_V_Af(x)+1./tauPH_EP_V_Af(x)+1./tauPH_PD_V_Af(x))    
    integrand(x)=tauPH_C_V_Af(x).*(x.^4).*exp(x)./(exp(x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,alphat)
    return a= do_quad(integrand,nodes, weights)   
end

function I2V(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)  
    tauPH_U_V_Af(x)=tauPH_U_VT(gammaSA,x,Tt,MSiGecgs,thetaD,omegaD,beta,delta)    
    tauPH_EP_V_Af(x)=tauPH_EP_VT(Eep*q,mds,x,etha,vtt,Tt,dd)/6    
    tauPH_PD_V_Af(x)=tauPH_PD_VT(GM,Tt,x,vtt,delta)    
    tauPH_C_V_Af(x)=1./((beta+1)./tauPH_U_V_Af(x)+1./tauPH_EP_V_Af(x)+1./tauPH_PD_V_Af(x))    
    integrand(x)=beta*tauPH_C_V_Af(x)./tauPH_U_V_Af(x).*(x.^4).*exp(x)./(exp(x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,thetaD/Tt)
    return a= do_quad(integrand,nodes, weights)    
end

function I3V(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)  
   alphat=thetaD/Tt
    tauPH_U_SA_Af(x)=tauPH_U_VT(gammaSA,x,Tt,MSiGecgs,thetaD,omegaD,beta,delta)    
    tauPH_EP_SA_Af(x)=tauPH_EP_VT(Eep*q,mds,x,etha,vtt,Tt,dd)/6
    tauPH_PD_SA_Af(x)=tauPH_PD_VT(GM,Tt,x,vtt,delta)
    tauPH_C_SA_Af(x)=1./((beta+1)./tauPH_U_SA_Af(x)+1./tauPH_EP_SA_Af(x)+1./tauPH_PD_SA_Af(x))    
    integrand(x)=beta*1./tauPH_U_SA_Af(x).*
    (1-beta*tauPH_C_SA_Af(x)./tauPH_U_SA_Af(x)).*(x.^4).*exp(x)./(exp(x)-1).^2    
    nodes, weights = qnwlege(1000,0.0,alphat)
    return a= do_quad(integrand,nodes, weights)    
end
function kltV(gammaSA,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    I1t=I1V(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    I2t=I2V(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    I3t=I3V(gammaSA,GM,Tt,MSiGecgs,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    return kB^4*Tt^3/2/pi/pi/hbar^3/vtt*(I1t+I2t.^2/I3t)    
end

###########################################################################################################


xxsp=71.3
xsp=(100-xxsp)/100
Eep=5.0
etha=0.96
thetaD=524.0#524.0
#gammaSA,GM,Tts[Tti],MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha

Tt=300.0#thetaD
gammaSA=0.91
beta=2.0

MSi=4.6637e-26 # kg
MGe=1.206e-25  # kg
VSi=2.0024e-29 #m3
VGe=2.2636e-29 #m3
MM=MSi*(1-xsp)+MGe*xsp
VV=VSi*(1-xsp)+VGe*xsp

eps=39
deltaSi=VSi^(1/3)
deltaGe=VGe^(1/3)
delta=deltaSi*(1-xsp)+deltaGe*xsp
deltacgs=delta*100
V=VSi*(1-xsp)+VGe*xsp
MSiGecgs(xsp)=28.084*(1-xsp)+xsp*72.64
G=1.033*(1-xsp)+1.017*xsp

a3=(2.7155^3)*(1-xsp)+(2.8288^3)*xsp
a1=a3^(1/3)
vtt=kB/hbar*(6*pi*pi)^(-1/3)*thetaD*delta
dd=(4.66e-26*(1-xsp)+1.206e-25*xsp)/(2e-29*(1-xsp)+2.2636e-29*xsp)

delMSiGe=28.084-72.64
deldelta=deltaSi-deltaGe
GM=xsp*(1-xsp)*((delMSiGe/MSiGecgs(xsp))^2+eps*(deldelta/delta)^2)

omegaD=thetaD*kB/hbar
int=collect(0.01:0.01:0.5)#collect(0.0001:0.001:1.0) # int=hbar*omega/kB/T
intx=int.*Tt/thetaD

mds=0.427*me
vt=5880.0
#println("deltaSi ",deltaSi,"    deltaGe ",deltaGe)

tauPH_U_SA_A=Array{Float64,1}(length(intx))

tauPH_U_SA_A=tauPH_U_SAT(gammaSA,intx,Tt,MSiGecgs(xsp),thetaD,omegaD,beta,delta)
tauPH_C_SA_A=tauPH_U_SA_A/(1+beta)

tauPH_U_V_A=tauPH_U_VT(gammaSA,int,Tt,MSiGecgs(xsp),thetaD,omegaD,beta,delta)
tauPH_EP_V_A=tauPH_EP_VT(Eep*q,mds,int,etha,vtt,Tt,dd)/6
tauPH_C_V_A=tauPH_U_V_A/(beta+1) #tauPH_C_VT(gammaSA,int,Tt,MSiGecgs(xsp),thetaD,omegaD,beta,delta)
#plot(intx,tauPH_U_SA_A.*intx.^2,Linestyle="-",color="k")


tauPH_EP_SA_A=tauPH_EP_SAT(Eep,mds,intx,etha,Tt,MSiGecgs(xsp),thetaD,delta)
tauPH_PD_SA_A=tauPH_PD_SAT(GM,intx,thetaD)
tauPH_CUNEP_SA_A=1./(1./tauPH_C_SA_A+1./tauPH_EP_SA_A)
tauPH_CUNPD_SA_A=1./(1./tauPH_C_SA_A+1./tauPH_PD_SA_A)
tauPH_CUNPDEP_SA_A=1./(1./tauPH_C_SA_A+1./tauPH_PD_SA_A+1./tauPH_EP_SA_A)
tauPH_e_A=tauPH_eT(Eep*q,mds,dd,etha,int,Tt,vtt)/6

tauPH_U_SS_A=tauPH_UT(gammaSA,int,Tt,MM,vtt,thetaD)
#tauPH_U_SS_A=tauPH_UT(0.3,int,50.01,6.786e-26,7.4226e3,586.0)
#tauPH_U_SAT(gammaSA,intx,Tt,MSiGecgs(xsp),thetaD,omegaD,beta,delta)

Ttt=300.0
I1_SA_A=I1(gammaSA,GM,Ttt,MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha)
I1_V_A=I1V(gammaSA,GM,Ttt,MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)
I1_SS_A=I1SS(gammaSA,GM,Ttt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
I2_SA_A=I2(gammaSA,GM,Ttt,MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha)
I2_V_A=I2V(gammaSA,GM,Ttt,MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)
I2_SS_A=I2SS(gammaSA,GM,Ttt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 
I3_SA_A=I3(gammaSA,GM,Ttt,MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha)
I3_V_A=I3V(gammaSA,GM,Ttt,MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)
I3_SS_A=I3SS(gammaSA,GM,Ttt,MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd) 

Tts=collect(300.0:50:1200.0)#[300.0,500.0,800.0]
klt_SA_A=Array{Float64,1}(length(Tts))
klt_V_A=Array{Float64,1}(length(Tts))
klt_SS_A=Array{Float64,1}(length(Tts))
for Tti in 1:length(Tts)
    klt_SA_A[Tti]=klt(gammaSA,GM,Tts[Tti],MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha)
    klt_V_A[Tti]=kltV(gammaSA,Tts[Tti],MSiGecgs(xsp),thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
    klt_SS_A[Tti]=kltSS(gammaSA,GM,Tts[Tti],MM,VV,thetaD,omegaD,beta,delta,Eep,mds,etha,vtt,dd)    
end

SiGe_SA_162_Bulk_KC_T=[298.201949929,399.8236245162,498.0843662731,598.5351036206,698.9858409681,797.1240997501,
897.5356425457,997.8589975993,1097.300475234,1199.132820538]
SiGe_SA_162_Bulk_KC_KL=[0.0394665342,0.0376127718,0.0362474716,0.0350157828,0.033784094,0.0335298893,0.0326537511,
0.0325776017,0.0305013403,0.0267364936]

figure()
plot(intx,tauPH_U_SA_A,Linestyle="-",color="r")
plot(intx,tauPH_U_SS_A,Linestyle="-",color="b")
grid()

figure()
title("kl")
plot(Tts,klt_SA_A,Linestyle="-",color="r")
plot(Tts,klt_V_A/100,Linestyle="-",color="b")
plot(Tts,klt_SS_A/100,Linestyle="-",color="k")
plot(SiGe_SA_162_Bulk_KC_T,SiGe_SA_162_Bulk_KC_KL,linestyle="None",marker="o",markersize=7,color="b")
#ylim(0.025,0.075)
grid()

figure()
title("Vinnig EP")
plot(intx,tauPH_EP_V_A.*intx.^2,Linestyle="-",color="c")
plot(intx,tauPH_EP_SA_A.*intx.^2,Linestyle="-",color="k")
plot(intx,tauPH_e_A.*intx.^2,Linestyle="-",color="y")
#plot(int,tauPH_EP_V_A,Linestyle="-",color="c")
#plot(intx,tauPH_EP_SA_A,Linestyle="-",color="k")
grid()

figure()
plot(intx,tauPH_C_SA_A.*intx.^2,Linestyle="-",color="c")
plot((intx),tauPH_CUNEP_SA_A.*(intx).^2,Linestyle="-",color="b")
plot((intx),tauPH_CUNPD_SA_A.*(intx).^2,Linestyle="-",color="k")
plot((intx),tauPH_CUNPDEP_SA_A.*(intx).^2,Linestyle="-",color="r")

#ylim(0.0,7e-13)
grid()

figure()
title("Vining")
kk=1e10*vtt
#plot((intx),kk*tauPH_C_V_A.*(intx).^2,Linestyle="-",color="b")
plot((intx),kk*tauPH_C_SA_A.*(intx).^2,Linestyle="-",color="b")
plot((intx),kk*tauPH_CUNEP_SA_A.*(intx).^2,Linestyle="-",color="b")
plot((intx),kk*tauPH_CUNPD_SA_A.*(intx).^2,Linestyle="-",color="k")
plot((intx),kk*tauPH_CUNPDEP_SA_A.*(intx).^2,Linestyle="-",color="r")
#ylim(0.0,12)
grid()

#figure()
#plot((intx),tauPH_EP_SA_A.*(intx).^2,Linestyle="-",color="b")
#grid()

#figure()
#plot((intx),tauPH_CUNEP_SAT_A.*(intx).^2,Linestyle="-",color="b")
#ylim(0.0,7e-13)
#grid()

LoadError: UndefVarError: qnwlege not defined

In [7]:
println(I1_SA_A," ",I1_V_A," ",I1_SS_A)
println(I2_SA_A," ",I2_V_A," ",I2_SS_A)
println(I3_SA_A," ",I3_V_A," ",I3_SS_A)

LoadError: UndefVarError: I1_SA_A not defined

In [8]:
vtt

4836.601828855511

In [9]:
foreach(i->println("U_V:",[i],", ",tauPH_U_V_A[i],
" U_SA",[i],", ",tauPH_U_SA_A[i]," U_SS",[i],", ",tauPH_U_SS_A[i],", ",
tauPH_U_V_A[i]/tauPH_U_SS_A[i]),1:length(tauPH_U_SA_A))

U_V:[1], 8.644808209850447e-8 U_SA[1], 8.647026821401672e-8 U_SS[1], 3.684453591832207e-7, 0.2346293146157271
U_V:[2], 2.1612020524626117e-8 U_SA[2], 2.161756705350418e-8 U_SS[2], 9.211133979580517e-8, 0.2346293146157271
U_V:[3], 9.605342455389386e-9 U_SA[3], 9.607807579335188e-9 U_SS[3], 4.093837324258008e-8, 0.2346293146157271
U_V:[4], 5.403005131156529e-9 U_SA[4], 5.404391763376045e-9 U_SS[4], 2.3027834948951293e-8, 0.2346293146157271
U_V:[5], 3.4579232839401778e-9 U_SA[5], 3.4588107285606686e-9 U_SS[5], 1.4737814367328826e-8, 0.23462931461572709
U_V:[6], 2.4013356138473465e-9 U_SA[6], 2.401951894833797e-9 U_SS[6], 1.023459331064502e-8, 0.2346293146157271
U_V:[7], 1.7642465734388663e-9 U_SA[7], 1.7646993513064628e-9 U_SS[7], 7.519293044555525e-9, 0.23462931461572706
U_V:[8], 1.3507512827891323e-9 U_SA[8], 1.3510979408440112e-9 U_SS[8], 5.756958737237823e-9, 0.2346293146157271
U_V:[9], 1.0672602728210429e-9 U_SA[9], 1.0675341754816876e-9 U_SS[9], 4.548708138064453e-9, 0.2346293146157

In [10]:
#foreach(i->println("tauPH_EP_V_A ",[i]," ",tauPH_EP_V_A[i],
#" tauPH_EP_SA_A",[i]," ",tauPH_EP_SA_A[i]," tauPH_e_A",[i]," ",tauPH_e_A[i]),1:length(tauPH_EP_SA_A))

In [11]:
foreach(i->println(Tts[i]," ",klt_SA_A[i],"  ",klt_V_A[i]),1:length(Tts))

LoadError: UndefVarError: Tts not defined

In [12]:
klt_A

LoadError: UndefVarError: klt_A not defined

In [13]:
klt_A

LoadError: UndefVarError: klt_A not defined

In [14]:
klt_A

LoadError: UndefVarError: klt_A not defined

In [15]:
(0.0162^2/3.488e11+4.64102e-11)*4.66367e11

21.644536641781535

In [16]:
(thetaD^2/delta/100)*4.67e-2

4.666168150877512e11

In [17]:
xspt=0.2
G=1.033*(1-xspt)+1.017*xspt
VSit=2.7155^3 #A^3
VGet=2.8288^3 #A^3
Vt=VSit*(1-xspt)+VGet*xspt #A^3
Vcur=Vt^(1/3) #A
println("VSit ",VSit," VGet ",VGet," Vt ",Vt)
a3t=(2.7155^3)*(1-xspt)+(2.8288^3)*xspt #A^3
a1t=a3t^(1/3) #A
println("a3t ",a3t," a1t ",a1t," Vcur ",Vcur)
println("Comparison a1 ",a1," delta ",delta)
Mt=28.086*(1-xspt)+72.59*xspt  #g/mol
println("Mt ",Mt)
dt=Mt/a3t #g/A^3
dtmks=dt/1000/Nav/(1e-30)
println("dt ",dt," dtmks ",dtmks, " dd ",dd)
Vtmks=(1-xspt)*VSit*1e-30+xspt*VGet*1e-30
Mtmks=(1-xspt)*4.6637e-26+xspt*1.206e-25
dtmks2=Mtmks/Vtmks
println("Vtmks ",Vtmks," Mtmks ",Mtmks," dtmks2 ",dtmks2)
thetaDt=524.0
vttt=kB/hbar*(6*pi*pi)^(-1/3)*thetaDt*a1t*1e-10 #m/s
println("vttt ",vttt," vtt ",vtt," vt ",vt)

VSit 20.023934748875 VGet 22.636367183872007 Vt 20.5464212358744
a3t 20.5464212358744 a1t 2.7389161019057777 Vcur 2.7389161019057777
Comparison a1 2.7489782587837404 delta 2.7480148133085564e-10
Mt 36.9868
dt 1.8001577781059224 dtmks 2989.231950305256 dd 3268.2723684284065
Vtmks 2.0546421235874404e-29 Mtmks 6.14296e-26 dtmks2 2989.7956094048564
vttt 4820.587779732564 vtt 4836.601828855511 vt 5880.0


In [18]:
(1.054e-27)^2*(6*pi*pi)^(2/3)/8/(9.109e-28)/1.38e-16

1.6783226027805357e-11

In [19]:
kbcgs=1.3807e-16
hbarcgs=1.054e-27
mecgs=9.109e-28
yy=(1/2*mecgs)*kbcgs/hbarcgs/hbarcgs*(6*pi*pi)^(-2/3)
DD=hbarcgs^2*(6*pi*pi)^(2/3)/8/mecgs/kbcgs
aa=kbcgs^3/(2*pi*pi)/(6*pi*pi)^(-1/3)/hbarcgs^2/1000/100/100
eee=kbcgs/hbarcgs*(6*pi)/4

6.173050726012467e11

In [20]:
aa

0.04678471738321031

In [21]:
vtt

4836.601828855511

In [22]:
foreach(i->println("tauPH_U_V_A ",[i]," ",tauPH_U_V_A[i],
" tauPH_U_SA_A",[i]," ",tauPH_U_SA_A[i]," tauPH_U_SS_A",[i]," ",tauPH_U_SS_A[i]),1:length(tauPH_U_SA_A))

tauPH_U_V_A [1] 8.644808209850447e-8 tauPH_U_SA_A[1] 8.647026821401672e-8 tauPH_U_SS_A[1] 3.684453591832207e-7
tauPH_U_V_A [2] 2.1612020524626117e-8 tauPH_U_SA_A[2] 2.161756705350418e-8 tauPH_U_SS_A[2] 9.211133979580517e-8
tauPH_U_V_A [3] 9.605342455389386e-9 tauPH_U_SA_A[3] 9.607807579335188e-9 tauPH_U_SS_A[3] 4.093837324258008e-8
tauPH_U_V_A [4] 5.403005131156529e-9 tauPH_U_SA_A[4] 5.404391763376045e-9 tauPH_U_SS_A[4] 2.3027834948951293e-8
tauPH_U_V_A [5] 3.4579232839401778e-9 tauPH_U_SA_A[5] 3.4588107285606686e-9 tauPH_U_SS_A[5] 1.4737814367328826e-8
tauPH_U_V_A [6] 2.4013356138473465e-9 tauPH_U_SA_A[6] 2.401951894833797e-9 tauPH_U_SS_A[6] 1.023459331064502e-8
tauPH_U_V_A [7] 1.7642465734388663e-9 tauPH_U_SA_A[7] 1.7646993513064628e-9 tauPH_U_SS_A[7] 7.519293044555525e-9
tauPH_U_V_A [8] 1.3507512827891323e-9 tauPH_U_SA_A[8] 1.3510979408440112e-9 tauPH_U_SS_A[8] 5.756958737237823e-9
tauPH_U_V_A [9] 1.0672602728210429e-9 tauPH_U_SA_A[9] 1.0675341754816876e-9 tauPH_U_SS_A[9] 4.54870813

In [23]:
tauPH_EP_V_A/tauPH_EP_SA_A

50×50 Array{Float64,2}:
 9.51387e-9  4.77974e-9   3.21241e-9   …  2.69704e-5  4.58735e-5  7.89511e-5
 4.77973e-9  2.40132e-9   1.6139e-9       1.35498e-5  2.30466e-5  3.96647e-5
 3.2124e-9   1.6139e-9    1.08469e-9      9.10667e-6  1.54894e-5  2.66582e-5
 2.43745e-9  1.22457e-9   8.2302e-10      6.90981e-6  1.17528e-5  2.02272e-5
 1.98027e-9  9.94879e-10  6.68649e-10     5.61376e-6  9.54834e-6  1.64333e-5
 1.6829e-9   8.45482e-10  5.6824e-10   …  4.77076e-6  8.1145e-6   1.39656e-5
 1.47792e-9  7.42504e-10  4.99029e-10     4.18969e-6  7.12617e-6  1.22646e-5
 1.33193e-9  6.69158e-10  4.49735e-10     3.77583e-6  6.42224e-6  1.10531e-5
 1.2267e-9   6.16289e-10  4.14201e-10     3.4775e-6   5.91482e-6  1.01798e-5
 1.15168e-9  5.78598e-10  3.8887e-10      3.26483e-6  5.55309e-6  9.55721e-6
 1.10063e-9  5.52951e-10  3.71633e-10  …  3.12011e-6  5.30694e-6  9.13357e-6
 1.06995e-9  5.3754e-10   3.61276e-10     3.03315e-6  5.15904e-6  8.87902e-6
 1.05786e-9  5.31464e-10  3.57192e-10     2.99887e-6

In [24]:
tauPH_EP_SA_A

50-element Array{Float64,1}:
 7.08446e-10
 3.55921e-10
 2.39211e-10
 1.81506e-10
 1.47462e-10
 1.2532e-10 
 1.10057e-10
 9.91871e-11
 9.13521e-11
 8.57674e-11
 8.1968e-11 
 7.96864e-11
 7.8789e-11 
 ⋮          
 2.8766e-8  
 4.39516e-8 
 6.79683e-8 
 1.06379e-7 
 1.68501e-7 
 2.70104e-7 
 4.38157e-7 
 7.19263e-7 
 1.19479e-6 
 2.00834e-6 
 3.41595e-6 
 5.87906e-6 

In [25]:
kB/hbar*(6*pi*pi)^(-1/3)*thetaD*delta

4836.601828855511

In [26]:
intx

50-element Array{Float64,1}:
 0.00572519
 0.0114504 
 0.0171756 
 0.0229008 
 0.028626  
 0.0343511 
 0.0400763 
 0.0458015 
 0.0515267 
 0.0572519 
 0.0629771 
 0.0687023 
 0.0744275 
 ⋮         
 0.223282  
 0.229008  
 0.234733  
 0.240458  
 0.246183  
 0.251908  
 0.257634  
 0.263359  
 0.269084  
 0.274809  
 0.280534  
 0.28626   

In [27]:
foreach(println,tauPH_EP_SAT_A)

LoadError: UndefVarError: tauPH_EP_SAT_A not defined

In [28]:
kB/hbar*(6*pi*pi)^(-1/3)*554.0*delta

5113.506513713651

In [29]:
foreach(println,tauPH_C_V_A)

2.8816027366168154e-8
7.2040068415420386e-9
3.2017808184631286e-9
1.8010017103855096e-9
1.152641094646726e-9
8.004452046157822e-10
5.880821911462887e-10
4.502504275963774e-10
3.55753424273681e-10
2.881602736616815e-10
2.381489864972575e-10
2.0011130115394554e-10
1.7050903766963403e-10
1.4702054778657218e-10
1.2807123273852514e-10
1.1256260689909435e-10
9.9709437253177e-11
8.893835606842024e-11
7.982279048800043e-11
7.204006841542037e-11
6.5342465682921e-11
5.953724662431437e-11
5.447264152394736e-11
5.0027825288486385e-11
4.610564378586905e-11
4.262725941740851e-11
3.9528158252631216e-11
3.6755136946643046e-11
3.4264004002578074e-11
3.2017808184631286e-11
2.998546031859329e-11
2.8140651724773588e-11
2.646099849969527e-11
2.492735931329425e-11
2.352328764585156e-11
2.223458901710506e-11
2.1048960822621004e-11
1.995569762200011e-11
1.894544862995934e-11
1.8010017103855093e-11
1.7142193555126807e-11
1.633561642073025e-11
1.558465514665666e-11
1.4884311656078592e-11
1.4230136970947238e-11


In [30]:
@time klattice=thermalPropsiso(tauPHL,tauPHTx,tauPHTx,[vSiGeL_PH(xsp),vSiGeT_PH(xsp),vSiGeT_PH(xsp)])
xmax=3.0

LoadError: UndefVarError: thermalPropsiso not defined

In [31]:
SiGe_Min_1700_Bulk_EC_T=[309.4827586207,389.6551724138,450,479.3103448276,504.3103448276,589.6551724138,979.3103448276,
    1039.6551724138,1110.3448275862,1189.6551724138,1199.1379310345]
SiGe_Min_1700_Bulk_EC_E=[1252.3364485981,1113.707165109,1054.5171339564,1054.5171339564,912.7725856698,834.8909657321,
    526.4797507788,434.5794392523,478.1931464174,557.6323987539,501.5576323988]
SiGe_Min_730_Bulk_EC_T=[308.6206896552,360.3448275862,419.8275862069,509.2672413793,609.2432950192,957.662835249,
    1009.1475095786,1098.9463601533,1149.2337164751,1218.0795019157]
SiGe_Min_730_Bulk_EC_E=[771.0280373832,716.5109034268,626.1682242991,556.7670474213,455.0882658359,277.6912426445,
        273.3644859813,241.9955001731,231.1786085151,213.8715818622]    
SiGe_Min_230_Bulk_EC_T=[299.7365900383,398.5153256705,500.8860153257,598.4674329502,708.6206896552,868.4626436782,
        979.8132183908,1019.3247126437,1088.1704980843,1148.0363984674]
SiGe_Min_230_Bulk_EC_E=[314.4686742818,280.9363101419,249.5673243337,217.1166493596,184.6659743856,155.460366909,
        130.5815160955,131.6632052613,126.2547594323,131.6632052613]
SiGe_Min_22_Bulk_EC_T=[309.9137931034,348.2279693487,389.535440613,569.1331417625,690.0622605364,808.596743295,
    909.7701149425,989.3917624521,1079.1906130268]
SiGe_Min_22_Bulk_EC_E=[49.4548286604,54.8632744894,44.0463828314,38.6379370024,31.0661128418,25.6576670128,24.575977847,
    31.0661128418,45.1280719972]
xmax=1
SiGe_Min_1700_Bulk_SC_T=[310.6382978723,450.2127659574,500.4255319149,579.5744680851,779.5744680851,970.2127659574,
    1050.2127659575,1100.4255319149,1151.4893617021,1159.1489361702]
SiGe_Min_1700_Bulk_SC_S=[-105.0763607884,-144.5542357395,-159.9686724971,-169.9060174912,-220.5195144237,-245.9652786842,
    -276.149327764,-276.2263412087,-260.3537397207,-240.1200887613]
SiGe_Min_730_Bulk_SC_T=[310.6382978723,361.7021276596,419.5744680851,509.7872340426,610.2127659574,1009.3617021277,
    1219.5744680851]
SiGe_Min_730_Bulk_SC_S=[-165.1990601749,-180.0013053126,-210.1514162642,-220.7192272549,-250.3211069051,-330.687899752,
    -346.3477352826]
SiGe_Min_230_Bulk_SC_T=[298.7234042553,400.8510638298,499.5744680851,600.8510638298,710.6382978723,859.5744680851,
    978.7234042553,1039.1489361702,1090.2127659575,1159.1489361702]
SiGe_Min_230_Bulk_SC_S=[-249.8433624853,-284.9693251534,-305.3661401906,-320.245398773,-335.7512074142,-342.1146064482,
-342.9108471479,-330.7335856938,-309.3395118131,-251.1630335465]
SiGe_Min_22_Bulk_SC_T=[309.7872340426,349.7872340426,380.4255319149,410.2127659574,550.6382978723,690.2127659574,
824.6808510638,910.6382978723,979.5744680851]
SiGe_Min_22_Bulk_SC_S=[-479.9216812427,-489.7989818562,-519.9073228038,-524.8609842057,-534.8923117087,-544.922333899,
-540.2205978332,-480.8432319541,-372.9735021538]
xmax=1
SiGe_Min_1700_Bulk_KC_T=[314.5631067961,444.0129449838,499.2448759439,577.7777777778,769.3635382956,966.1272923409,
    1032.5782092772,1105.9331175836,1168.932038835,1188.7810140237]
SiGe_Min_1700_Bulk_KC_K=[4.6835063051,4.5239984377,4.4312520924,4.3076219172,4.4329985493,4.1911672804,4.0622865752,
4.2179332664,4.4666164491,4.3891585761]
SiGe_Min_22_Bulk_KC_T=[314.5631067961,378.4250269687,554.476806904,683.9266450917,814.2394822006,903.9913700108,
    973.0312837109]
SiGe_Min_22_Bulk_KC_K=[7.1197132017,6.4011605848,5.1350574713,4.6289978797,4.3608749024,4.4855931258,4.7291429528]
SiGe_Vin_162_Bulk_KCL_T=[440.3713548665536,497.3222430383224,574.0370374003198]
SiGe_Vin_162_Bulk_KCL_KL=[3.3457249070631976,3.2416356877323427,3.074349442379183]

SiGe_Su_162_Bulk_KCL_T=[51.5096343499,56.6540954115,60.7039902898,64.5349719314,68.5191928387,73.5104146347,78.545419078,
84.2006776918,89.8194507662,96.7152177211,105.1433773327,115.103929601,126.5968745259,140.3884084357,156.4785313306,
173.3348505538,190.191169777,207.0474890002,223.9038082233,240.7601274465,257.6164466697,274.4727658929,291.3290851161,
308.1854043393,325.0417235624,341.8980427856,358.7543620088,375.610681232,392.4670004552,409.3233196784,424.6472462449,
470.6190259445,487.4753451677,504.3316643908,521.187983614,538.0443028372,554.9006220604,571.7569412836,588.6132605068,
605.4695797299,622.3258989531,639.1822181763,656.0385373995,672.8948566227,689.7511758459,706.607495069,723.4638142922,
    740.3201335154,757.1764527386,774.0327719618,438.2642998028,452.4654832347]
SiGe_Su_162_Bulk_KCL_KL=[1.912952409,2.0367544267,2.1416796981,2.2300607933,2.316951567,2.4101428367,2.4960198089,
2.5805877859,2.6667920069,2.747450222,2.8331721816,2.9171373319,2.9981959963,3.0776786993,3.1519579117,3.2134462372,
3.2637548671,3.3065007618,3.3406974776,3.3686467164,3.3926501804,3.4110637965,3.4258604524,3.4370401479,3.443945254,
3.4469045852,3.4531520621,3.4482198435,3.4452605123,3.4409859228,3.4325828837,3.4025146176,3.3880467763,3.3719348622,
3.355822948,3.3383957755,3.3196533448,3.3002532849,3.2795379667,3.2575073902,3.2371208866,3.2157479392,3.1940461773,
3.1713579716,3.1483409514,3.1253239312,3.102306911,3.0792898907,3.0549576122,3.031940592,3.426443203,3.4189944134]

SiGe_Dis_162_Bulk_KC_T=[323.4130104234,436.3766249262,465.5560008689,494.6582798193,568.7099409229,676.8421439083,
    861.4563988974,997.8960475554,1141.7663777569,1201.6652338456]
SiGe_Dis_162_Bulk_KC_K=[4.6738529713,4.5204119777,4.5006535608,4.430757327,4.3128769847,4.2031161293,3.9957878466,
4.065915243,4.2121420299,4.3719065138]

SiGe_SA_162_Bulk_KC_T=[398.7219168438,498.0855086789,598.5416158272,698.9878915666,797.1262440682,897.5381098765,
997.8565773003,1097.3037361109,1199.1374694305]
SiGe_SA_162_Bulk_KC_KL=[3.7593499332,3.62237734,3.494337284,3.3752157204,3.3497102368,3.2618033964,3.2586222334,
3.0458424552,2.6680986582]

figure()
plot(Ts,sigmas[:,1,1]/100,linestyle="-",color="b")
plot(Ts,sigmas[:,2,2]/100,linestyle="-",color="r")
plot(Ts,sigmas[:,3,3]/100,linestyle="-",color="g")
plot(Ts,sigmas[:,4,4]/100,linestyle="-",color="#ff66cc") 
plot(SiGe_Min_1700_Bulk_EC_T,SiGe_Min_1700_Bulk_EC_E, linestyle="None",marker="o",markersize=7,color="b")
plot(SiGe_Min_730_Bulk_EC_T,SiGe_Min_730_Bulk_EC_E, linestyle="None",marker="s",markersize=7,color="r")
plot(SiGe_Min_230_Bulk_EC_T,SiGe_Min_230_Bulk_EC_E, linestyle="None",marker="^",markersize=7,color="g")
plot(SiGe_Min_22_Bulk_EC_T,SiGe_Min_22_Bulk_EC_E, linestyle="None",marker="D",markersize=7,color="#ff66cc")
ylim(0,1500)
grid()

figure()
plot(Ts,seebecks[:,1,1]*1e6,linestyle="-",color="b")
plot(Ts,seebecks[:,2,2]*1e6,linestyle="-",color="r")
plot(Ts,seebecks[:,3,3]*1e6,linestyle="-",color="g")
plot(Ts,seebecks[:,4,4]*1e6,linestyle="-",color="#ff66cc") 
plot(SiGe_Min_1700_Bulk_SC_T,SiGe_Min_1700_Bulk_SC_S, linestyle="None",marker="o",markersize=7,color="b")
plot(SiGe_Min_730_Bulk_SC_T,SiGe_Min_730_Bulk_SC_S, linestyle="None",marker="s",markersize=7,color="r")
plot(SiGe_Min_230_Bulk_SC_T,SiGe_Min_230_Bulk_SC_S, linestyle="None",marker="^",markersize=7,color="g")
plot(SiGe_Min_22_Bulk_SC_T,SiGe_Min_22_Bulk_SC_S, linestyle="None",marker="D",markersize=7,color="#ff66cc")
ylim(-600,0)
grid()

figure()
plot(Ts,klattice[:,1,1]+kes[:,1,1],linestyle="-",color="b")
#plot(Ts,klattice[:,2,2]+kes[:,2,2],linestyle="-",color="r")
#plot(Ts,klattice[:,3,3]+kes[:,3,3],linestyle="-",color="g")
plot(Ts,klattice[:,4,4]+kes[:,4,4],linestyle="-",color="#ff66cc") 
plot(SiGe_Min_1700_Bulk_KC_T,SiGe_Min_1700_Bulk_KC_K, linestyle="None",marker="o",markersize=7,color="b")
#plot(SiGe_Min_1700_Bulk_KC_T,SiGe_Min_1700_Bulk_KC_K,linestyle="None",marker="o",markersize=7,color="b")
#plot(SiGe_Min_730_Bulk_KC_T,SiGe_Min_730_Bulk_SC_S, linestyle="None",marker="s",markersize=7,color="r")
#plot(SiGe_Min_230_Bulk_SC_T,SiGe_Min_230_Bulk_SC_S, linestyle="None",marker="^",markersize=7,color="g")
plot(SiGe_Min_22_Bulk_KC_T,SiGe_Min_22_Bulk_KC_K, linestyle="None",marker="D",markersize=7,color="#ff66cc")
plot(SiGe_Dis_162_Bulk_KC_T,SiGe_Dis_162_Bulk_KC_K, linestyle="None",marker="D",markersize=7,color="k")
ylim(3,8)
grid()

figure()
plot(Ts,klattice[:,1,1],linestyle="-.",color="b",label="kl")
#plot(Ts,klattice[:,2,2],linestyle="-.",color="r")
#plot(Ts,klattice[:,3,3],linestyle="-.",color="g")
#plot(Ts,klattice[:,4,4],linestyle="-.",color="#ff66cc") 
plot(SiGe_Vin_162_Bulk_KCL_T,SiGe_Vin_162_Bulk_KCL_KL, linestyle="None",marker="s",markersize=4,color="c")
plot(SiGe_Su_162_Bulk_KCL_T,SiGe_Su_162_Bulk_KCL_KL, linestyle="None",marker="s",markersize=4,color="c")
plot(SiGe_SA_162_Bulk_KC_T,SiGe_SA_162_Bulk_KC_KL,linestyle="None",marker="o",markersize=7,color="b")
plot(Ts,kes[:,1,1],linestyle="--",color="b",label="ke")
plot(Ts,kees[:,1,1],linestyle="--",color="k",label="kee")
plot(Ts,kehs[:,1,1],linestyle="--",color="y",label="keh")
#plot(Ts,kes[:,2,2],linestyle="--",color="r")
#plot(Ts,kes[:,3,3],linestyle="--",color="g")
#plot(Ts,kes[:,4,4],linestyle="--",color="#ff66cc") 
plot(Ts,kbis[:,1,1],linestyle=":",color="b",label="kbis")
#plot(Ts,kbis[:,2,2],linestyle=":",color="r")
#plot(Ts,kbis[:,3,3],linestyle=":",color="g")
#plot(Ts,kbis[:,4,4],linestyle=":",color="#ff66cc") 
#ylim()
legend(loc=1)
grid()
#plot(SiGe_Vin_162_Bulk_KC_T,SiGe_Vin_162_Bulk_KC_K, linestyle="None",marker="s",markersize=4,color="c")
#plot(Ts,kl_A,Linestyle="-",Linewidth="2",color="b")




LoadError: UndefVarError: figure not defined

In [32]:
#kehs